In [ ]:
import pandas as pd
import numpy as np

In [ ]:
### change the raw data file

df = pd.read_csv('C:/Users/anaik/Downloads/nov_w2_gss_sales_v2.csv')

In [ ]:
df.drop(df.index[-1], inplace = True)
df['Store'] = df['Store'].astype(str).str.strip()

In [ ]:
df = df[df['Store'].isin(['Game Show Studio - Fort Worth', 
    'Game Show Studio - Sacramento', 
    'Game Show Studio - San Diego'])]

df['Store'].unique()

In [ ]:
df.loc[:, 'Store'] = df['Store'].apply(
    lambda x: x.replace('Game Show Studio - ', '') + ' - GSS'
    if isinstance(x, str) and 'Game Show Studio - ' in x else x
)

In [ ]:
df.loc[:, 'Store'] = df['Store'].str.strip()

In [ ]:
### change the raw data file

df1 = pd.read_csv('C:/Users/anaik/Downloads/nov_w2_itemized_sales_v2.csv')

In [ ]:
df1.drop(df1.index[-1], inplace = True)
df1['Store'] = df1['Store'].astype(str).str.strip()

In [ ]:
df1 = df1[df1['Store'] != '(Sandbox) Red Door']
df1['Store'].unique()

In [ ]:
df1.loc[:, 'Store'] = df1['Store'].apply(
    lambda x: x.replace('Red Door -', '')
    if isinstance(x, str) and ('Red Door - ') in x else x
)

In [ ]:
df1.loc[:,'Store'] = df1['Store'].str.replace(r'OKC\s*\(|\)', '', regex=True).str.strip()

In [ ]:
df_combined_IS = pd.concat([df, df1], ignore_index = True)
df_combined_IS['Store'].unique()

In [ ]:
df_combined_IS.rename(columns={'Store':'Location'}, inplace=True)

In [ ]:
df_combined_IS['Booking Total Taxes'] = pd.to_numeric(df_combined_IS['Booking Total Taxes'], errors='coerce').fillna(0)
df_combined_IS['Booking Total Custom Fees'] = pd.to_numeric(df_combined_IS['Booking Total Custom Fees'], errors='coerce').fillna(0)

In [ ]:
df_combined_IS = df_combined_IS.sort_values(by=['Location', 'ID'], ascending=[True, True])

In [ ]:
df_combined_IS['ID'] = df_combined_IS['ID'].astype(str).str.replace('\.0$', '', regex=True)
df_combined_IS['Booking ID'] = df_combined_IS['Booking ID'].astype(str).str.replace('\.0$', '', regex=True)

In [ ]:
pd.set_option('display.max_rows', None)
print(df_combined_IS[['ID', 'Booking Start Date', 'Booking Start Time']].head(50))
pd.reset_option('display.max_rows')

In [ ]:
df_combined_IS['Booking Start Date'] = df_combined_IS.groupby(['ID', 'Location'])['Booking Start Date'].ffill()
df_combined_IS['Booking Start Time'] = df_combined_IS.groupby(['ID', 'Location'])['Booking Start Time'].ffill()

df_combined_IS['Booking Start Date'] = df_combined_IS['Booking Start Date'].fillna(df_combined_IS['Date'])
df_combined_IS['Booking Start Time'] = df_combined_IS['Booking Start Time'].fillna(df_combined_IS['Time'])

# df_combined_IS[['Booking Start Date', 'Booking Start Time']] = (
#     df_combined_IS.groupby('ID')[['Booking Start Date', 'Booking Start Time']].ffill()
# )

In [ ]:
pd.set_option('display.max_rows', None)
print(df_combined_IS[['ID', 'Booking Start Date', 'Booking Start Time']].head(50))
pd.reset_option('display.max_rows')

In [ ]:
df_combined_IS['time id'] = df_combined_IS['Booking Start Date'].astype(str) + df_combined_IS['Booking Start Time'].astype(str)

In [ ]:
# List of keywords
keywords = ['game show', 'gameshow']

# Create regex pattern
pattern = '|'.join(keywords)  # 'game show studio|gameshow'

# Boolean mask for rows where Inventory Item contains any keyword
mask = df_combined_IS['Inventory Item'].str.contains(pattern, case=False, na=False)

# Append ' - GSS' only where mask is True and not already appended
df_combined_IS.loc[mask & ~df_combined_IS['Location'].str.endswith(' - GSS'), 'Location'] += ' - GSS'

In [ ]:
varchar_columns_IS = ['location code', 'loc id', 'reporting group', 'exclude']
numeric_columns_IS = ['taxes']

for col in varchar_columns_IS:
    df_combined_IS[col] = pd.Series(['']*len(df_combined_IS), dtype="string")

for col in numeric_columns_IS:
    df_combined_IS[col] = pd.Series([np.nan]*len(df_combined_IS), dtype="float")

In [ ]:
df_combined_IS['Discount Codes'] = df_combined_IS['Discount Codes'].str.slice(0,50)

In [ ]:
columns_to_keep_IS = ['Location', 'ID', 'Date', 'Time', 'Status', 'Payment Status', 'Booking ID', 'Booking Status', 
                   'Inventory Type', 'Inventory Item', 'Booking Start Date', 'Booking Start Time', 'Quantity', 'Subtotal', 
                   'Discount Codes Value', 'Gift Voucher Codes Value', 'Discount Codes', 'Booking Net Total', 'Booking Total Taxes',
                   'Booking Total Custom Fees', 'Total', 'Created by', 'Source', 'Initial Payment Method', 'time id', 'location code', 
                   'loc id', 'taxes', 'reporting group', 'exclude']

df_combined_IS = df_combined_IS[columns_to_keep_IS]

In [ ]:
### change the name of the output file

df_combined_IS.to_csv('C:/Users/anaik/Downloads/nov_w2_sales_cleaned_v4.csv', index=False, float_format='%.2f')

In [ ]:
### change the raw data file

df2 = pd.read_csv('C:/Users/anaik/Downloads/nov_w2_gss_transactions.csv')

In [ ]:
df2.drop(df2.index[-1], inplace = True)
df2['Store'] = df2['Store'].astype(str).str.strip()

In [ ]:
df2 = df2[df2['Store'].isin(['Game Show Studio - Fort Worth', 
    'Game Show Studio - Sacramento', 
    'Game Show Studio - San Diego'])]

df2['Store'].unique()

In [ ]:
df2.loc[:, 'Store'] = df2['Store'].apply(
    lambda x: f"{x.replace('Game Show Studio - ', '')} - GSS"
    if isinstance(x, str) and 'Game Show Studio - ' in x else x
)

In [ ]:
df2.loc[:, 'Store'] = df2['Store'].str.strip()

In [ ]:
df2.reset_index(drop=True, inplace=True)

In [ ]:
### change the raw data file

df3 = pd.read_csv('C:/Users/anaik/Downloads/nov_w2_all_transactions.csv')

In [ ]:
df3.drop(df3.index[-1], inplace = True)
df3['Store'] = df3['Store'].astype(str).str.strip()

In [ ]:
df3 = df3[df3['Store'] != '(Sandbox) Red Door']

In [ ]:
df3.loc[:, 'Store'] = df3['Store'].apply(
    lambda x: x.replace('Red Door -', '')
    if isinstance(x, str) and ('Red Door - ') in x else x
)

In [ ]:
df3.loc[:,'Store'] = df3['Store'].str.replace(r'OKC\s*\(|\)', '', regex=True).str.strip()

In [ ]:
df3['Store'].unique()

In [ ]:
df_combined_AT = pd.concat([df2, df3], ignore_index = True)
df_combined_AT['Store'].unique()

In [ ]:
df_combined_AT.rename(columns={'Store':'Location'}, inplace=True)

In [ ]:
df_combined_AT.rename(columns={'Transaction ID':'ID'}, inplace=True)
df_combined_AT['ID'] = df_combined_AT['ID'].astype(str).str.replace('\.0$', '', regex=True)

In [ ]:
df_combined_IS_first_item = df_combined_IS.sort_values(by=['Location', 'ID']).drop_duplicates(subset=['Location', 'ID'], keep='first')

df_combined_AT = df_combined_AT.merge(
    df_combined_IS_first_item[['Location', 'ID', 'Inventory Item']],
    on=['Location', 'ID'],
    how='left'
)

In [ ]:
# List of keywords
keywords = ['game show', 'gameshow']

# Create regex pattern
pattern = '|'.join(keywords)  # 'game show studio|gameshow'

# Boolean mask for rows where Inventory Item contains any keyword
mask = df_combined_AT['Inventory Item'].str.contains(pattern, case=False, na=False)

# Append ' - GSS' only where mask is True and not already appended
df_combined_AT.loc[mask & ~df_combined_AT['Location'].str.endswith(' - GSS'), 'Location'] += ' - GSS'

In [ ]:
varchar_columns_AT = ['location code', 'loc id']
numeric_columns_AT = ['new total paid', 'no negatives' 'minus taxes fees', 'revenue calculated']

for col in varchar_columns_AT:
    df_combined_AT[col] = pd.Series(['']*len(df_combined_AT), dtype="string")

for col in numeric_columns_AT:
    df_combined_AT[col] = pd.Series([np.nan]*len(df_combined_AT), dtype="float")

In [ ]:
df_combined_AT['new total paid'] = df_combined_AT['Total Paid'] - df_combined_AT['Gift Vouchers Redeemed Value']

df_combined_AT['no negatives'] = np.where(df_combined_AT['new total paid'] > 0, 
                                          df_combined_AT['new total paid'], 0)

df_combined_AT['minus taxes fees'] = df_combined_AT['no negatives'] - (df_combined_AT['Total Taxes'] + 
                                                                       df_combined_AT['Total Custom Fees'])

df_combined_AT['revenue calculated'] = np.where(df_combined_AT['minus taxes fees'] > 0, 
                                                df_combined_AT['minus taxes fees'], 
                                                df_combined_AT['no negatives'])


float_cols = ['new total paid', 'no negatives', 'minus taxes fees', 'revenue calculated']
df_combined_AT[float_cols] = df_combined_AT[float_cols].astype(float)

In [ ]:
columns_to_keep_AT = ['Location', 'ID', 'Inventory Item', 'Transaction Date', 'Transaction Status', 'No. of Bookings', 'Bookings Value', 
                      'No. of Gift Vouchers', 'Gift Vouchers Value', 'Transaction Subtotal', 'Discounts Value', 'Discount Codes', 
                      'Gift Vouchers Redeemed Value', 'Gift Vouchers Applied', 'Transaction Net Total', 'Total Taxes', 'Total Custom Fees', 
                      'Transaction Total', 'Transaction Payment Status', 'Total Paid', 'Total Unpaid', 'Total Overpaid', 'Source', 'Created By', 
                      'new total paid', 'no negatives', 'minus taxes fees', 'revenue calculated', 'location code', 'loc id']

df_combined_AT = df_combined_AT[columns_to_keep_AT]

In [ ]:
### change the name of the output file

df_combined_AT.to_csv('C:/Users/anaik/Downloads/nov_w2_transactions_cleaned.csv', index=False, float_format='%.2f')

In [ ]:
# pip install sqlalchemy psycopg2-binary

In [ ]:
# from sqlalchemy import create_engine

In [ ]:
# username = 'postgres_admin'
# password = 'Reddoor1984!!!'
# host = 'scott-postgres-db.clf9wpharhk0.us-east-2.rds.amazonaws.com'
# port = '5432'
# database = 'reddoor-warehouse'

# engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

In [ ]:
# float_cols_IS = df_combined_IS.select_dtypes(include='float').columns
# float_cols_AT = df_combined_AT.select_dtypes(include='float').columns

# df_combined_IS[float_cols_IS] = df_combined_IS[float_cols_IS].round(2)
# df_combined_AT[float_cols_AT] = df_combined_AT[float_cols_AT].round(2)

In [ ]:
# df_combined_IS.to_sql('df_combined_is_sample', engine, schema='cashflow_revenue_xola', if_exists='replace', index=False)
# df_combined_AT.to_sql('df_combined_at_sample', engine, schema='cashflow_revenue_xola', if_exists='replace', index=False)

In [ ]:
### change the raw data file

df4 = pd.read_csv('C:/Users/anaik/Downloads/oct1_nov9_gss_payments.csv')

In [ ]:
df4.drop(df4.index[-1], inplace = True)
df4['Store'] = df4['Store'].astype(str).str.strip()

In [ ]:
df4 = df4[df4['Store'].isin(['Game Show Studio - Fort Worth', 
    'Game Show Studio - Sacramento', 
    'Game Show Studio - San Diego'])]

df4['Store'].unique()

In [ ]:
df4.loc[:, 'Store'] = df4['Store'].apply(
    lambda x: x.replace('Game Show Studio - ', '') + ' - GSS'
    if isinstance(x, str) and 'Game Show Studio - ' in x else x
)

In [ ]:
df4.loc[:, 'Store'] = df4['Store'].str.strip()

In [ ]:
### change the raw data file

df5 = pd.read_csv('C:/Users/anaik/Downloads/oct1_nov9_er_payments.csv')

In [ ]:
df5.drop(df5.index[-1], inplace = True)
df5['Store'] = df5['Store'].astype(str).str.strip()

In [ ]:
df5 = df5[df5['Store'] != '(Sandbox) Red Door']
df5['Store'].unique()

In [ ]:
df5.loc[:, 'Store'] = df5['Store'].apply(
    lambda x: x.replace('Red Door -', '')
    if isinstance(x, str) and ('Red Door - ') in x else x
)

In [ ]:
df5.loc[:,'Store'] = df5['Store'].str.replace(r'OKC\s*\(|\)', '', regex=True).str.strip()

In [ ]:
df_combined_PR = pd.concat([df4, df5], ignore_index = True)
df_combined_PR['Store'].unique()

In [ ]:
df_combined_PR.rename(columns={'Store':'Location'}, inplace=True)

In [ ]:
varchar_columns_PR = ['location code', 'loc id']

for col in varchar_columns_PR:
    df_combined_PR[col] = pd.Series(['']*len(df_combined_PR), dtype="string")

In [ ]:
columns_to_keep_PR = ['Location', 'Transaction ID', 'Transaction Date', 'Transaction Status', 'Transaction Payment Status', 
                      'Transaction Total Value', 'Transaction Value Paid', 'Transaction Value Remaining', 'Date', 'Type', 'Method', 
                      'Value', 'Refunded Value', 'Remaining Value', 'Refund as Customer Credit', 'Applied By', 'location code', 'loc id']

df_combined_PR = df_combined_PR[columns_to_keep_PR]

In [ ]:
### change the name of the output file

df_combined_PR.to_csv('C:/Users/anaik/Downloads/oct1_nov9_payments_cleaned.csv', index=False, float_format='%.2f')